In [1]:
import couchdb

In [2]:
try:
    couch = couchdb.Server('http://115.146.93.140:5984/')
    db = couch['raw_tweets']
except:
    print 'Database Connection Error'

In [3]:
db['2d368c427b8e1c5c54ca382c290005da'].keys()

[u'favorited',
 u'contributors',
 u'truncated',
 u'text',
 u'_rev',
 u'is_quote_status',
 u'in_reply_to_status_id',
 u'user',
 u'filter_level',
 u'geo',
 u'id',
 u'favorite_count',
 u'lang',
 u'_id',
 u'entities',
 u'in_reply_to_user_id_str',
 u'retweeted',
 u'coordinates',
 u'timestamp_ms',
 u'source',
 u'in_reply_to_status_id_str',
 u'in_reply_to_screen_name',
 u'in_reply_to_user_id',
 u'place',
 u'retweet_count',
 u'created_at',
 u'id_str']

In [4]:
db['2d368c427b8e1c5c54ca382c290005da']['created_at']

u'Wed Apr 26 12:45:17 +0000 2017'

In [15]:
print db['2d368c427b8e1c5c54ca382c290005da']['geo'] is None
print db['2d368c427b8e1c5c54ca382c29004834']['geo'] is None

True
False


In [20]:
db['2d368c427b8e1c5c54ca382c29004834']['coordinates']

{u'coordinates': [145.00313, -37.93526], u'type': u'Point'}

In [28]:
db['2d368c427b8e1c5c54ca382c290005da']['created_at']

u'Wed Apr 26 12:45:17 +0000 2017'

In [31]:
import time
time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(db['2d368c427b8e1c5c54ca382c290005da']['created_at'],'%a %b %d %H:%M:%S +0000 %Y'))

'2017-04-26 12:45:17'

In [39]:
db['1548bdd4c221cba8108949df2f020713']['text']

u'Shhh! https://t.co/JkDHHCAcwj'

In [40]:
db['1548bdd4c221cba8108949df2f020713']['text']=1

In [41]:
db['1548bdd4c221cba8108949df2f020713']['text']

u'Shhh! https://t.co/JkDHHCAcwj'

In [52]:
db['1548bdd4c221cba8108949df2f020713']['_rev']

u'1-28960598628d615b82a474876e6151ca'

In [53]:
try:
    couch2 = couchdb.Server('http://115.146.93.140:5984/')
    db2 = couch2['raw_tweets']
except:
    #print 'Database Connection Error'
    #print >> f1, "Database Connection Error"
    #f1.write('DB error')
    logPrint('DB Error')

In [54]:
db2.save(db['1548bdd4c221cba8108949df2f020713'])

(u'1548bdd4c221cba8108949df2f020713', u'2-59239a8a7d9a99c9d9b622238950e7d8')

In [55]:
db['1548bdd4c221cba8108949df2f020713']['sentiment']=1

In [56]:
db['1548bdd4c221cba8108949df2f020713']['sentiment']

u'Neutral'

In [8]:
from couchdb.design import ViewDefinition

In [9]:
#ViewDefinition.sync_many

In [37]:
for row in db.view('views/TweetsbyHour',group=True,startkey=5):
    print row

<Row key=5, value=10855>
<Row key=6, value=11352>
<Row key=7, value=11209>
<Row key=8, value=10916>
<Row key=9, value=10959>
<Row key=10, value=10845>
<Row key=11, value=10164>
<Row key=12, value=9355>
<Row key=13, value=6001>
<Row key=14, value=4834>
<Row key=15, value=3538>
<Row key=16, value=2893>
<Row key=17, value=2625>
<Row key=18, value=2470>
<Row key=19, value=2517>
<Row key=20, value=3709>
<Row key=21, value=7132>
<Row key=22, value=9042>
<Row key=23, value=9539>


In [14]:
?db.view('TweetsbyHour')